In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('data/Seismic-Bumps.csv', header = None)
dataset.shape

(2584, 19)

In [3]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,a,a,N,15180,48,-72,-72,a,0,0,0,0,0,0,0,0,0,0,0
1,a,a,N,14720,33,-70,-79,a,1,0,1,0,0,0,0,0,2000,2000,0
2,a,a,N,8050,30,-81,-78,a,0,0,0,0,0,0,0,0,0,0,0
3,a,a,N,28820,171,-23,40,a,1,0,1,0,0,0,0,0,3000,3000,0
4,a,a,N,12640,57,-63,-52,a,0,0,0,0,0,0,0,0,0,0,0


## Create X and Y

In [4]:
X = dataset.iloc[:, 0:18].values
Y = dataset.iloc[:, 18].values


In [5]:
X.shape

(2584, 18)

In [6]:
Y.shape

(2584,)

In [7]:
X[0]

array(['a', 'a', 'N', 15180, 48, -72, -72, 'a', 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=object)

In [8]:
Y[0]

0

## Preprocess the Data

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [10]:
le_Y = LabelEncoder()

In [11]:
Y = le_Y.fit_transform(Y)
Y[0]

0

In [12]:
def encoder_X(index):
    le_X = LabelEncoder()
    X[:, index] = le_X.fit_transform(X[:, index])


In [13]:
to_be_encoded_indices = [0, 1, 2, 7]

In [14]:
for x in to_be_encoded_indices:
    encoder_X(x)

In [15]:
X[0]

array([0, 0, 0, 15180, 48, -72, -72, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=object)

In [16]:
ohe_X = OneHotEncoder(categorical_features = [7])

In [17]:
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]
X.shape

(2584, 19)

In [18]:
ohe_X = OneHotEncoder(categorical_features = [3])

In [19]:
X = ohe_X.fit_transform(X).toarray()
X = X[:, 1:]
X.shape

(2584, 20)

In [20]:
sc_X = StandardScaler()


In [21]:
X = sc_X.fit_transform(X)
X[0]


array([-0.76630515, -0.13757705, -0.29895828, -0.10838026, -0.73230209,
       -1.34374329, -0.32756058, -0.87207336, -1.05071094, -1.21145926,
       -0.62998398, -0.50225321, -0.51042346, -0.24273572, -0.06830542,
        0.        ,  0.        ,  0.        , -0.24332671, -0.22108685])

## Create Train and Test Data


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [24]:
X_train.shape

(2067, 20)

In [25]:
X_test.shape

(517, 20)

In [26]:
Y_train.shape

(2067,)

In [27]:
Y_test.shape

(517,)

In [28]:
pd.DataFrame(pd.DataFrame(Y_train)[0].value_counts())

,0
0,1935
1,132


In [29]:
pd.DataFrame(pd.DataFrame(Y_test)[0].value_counts())

,0
0,479
1,38


## Create and train the Neural Network Classifier

In [30]:

from keras.models import Sequential
from keras.layers import Dense

In [31]:
clf_ann = Sequential()

In [32]:
# 10 20 40 80 160
# 40 40 160

# First Hidden Layer
clf_ann.add(Dense(output_dim = 40, init = 'uniform', activation = 'relu', input_dim = 20))

clf_ann.add(Dense(output_dim = 40, init = 'uniform', activation = 'relu'))

clf_ann.add(Dense(output_dim = 160, init = 'uniform', activation = 'relu'))

# Output Layer
clf_ann.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compile the ANN
clf_ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN on the Train Data
clf_ann.fit(X_train, Y_train, batch_size = 10, nb_epoch = 100)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=20, units=40, kernel_initializer="uniform")`
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=40, kernel_initializer="uniform")`
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=160, kernel_initializer="uniform")`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epoch

Epoch 1/100
2067/2067 [==============================] - 1s 504us/step - loss: 0.2892 - acc: 0.9352
Epoch 2/100
2067/2067 [==============================] - 0s 174us/step - loss: 0.2115 - acc: 0.9361
Epoch 3/100
2067/2067 [==============================] - 0s 194us/step - loss: 0.2083 - acc: 0.9361
Epoch 4/100
2067/2067 [==============================] - 0s 216us/step - loss: 0.2052 - acc: 0.9361
Epoch 5/100
2067/2067 [==============================] - 0s 158us/step - loss: 0.2049 - acc: 0.9361
Epoch 6/100
2067/2067 [==============================] - 0s 140us/step - loss: 0.2029 - acc: 0.9361
Epoch 7/100
2067/2067 [==============================] - 0s 141us/step - loss: 0.2015 - acc: 0.9361
Epoch 8/100
2067/2067 [==============================] - 0s 174us/step - loss: 0.2022 - acc: 0.9361
Epoch 9/100
2067/2067 [==============================] - 0s 143us/step - loss: 0.1988 - acc: 0.9361
Epoch 10/100
2067/2067 [==============================] - 0s 138us/step - loss: 0.1980 - acc: 0.9361

2067/2067 [==============================] - 0s 164us/step - loss: 0.1236 - acc: 0.9569
Epoch 82/100
2067/2067 [==============================] - 0s 150us/step - loss: 0.1184 - acc: 0.9569
Epoch 83/100
2067/2067 [==============================] - 0s 158us/step - loss: 0.1185 - acc: 0.9608
Epoch 84/100
2067/2067 [==============================] - 0s 157us/step - loss: 0.1133 - acc: 0.9598
Epoch 85/100
2067/2067 [==============================] - 0s 149us/step - loss: 0.1159 - acc: 0.9613
Epoch 86/100
2067/2067 [==============================] - 0s 148us/step - loss: 0.1191 - acc: 0.9618
Epoch 87/100
2067/2067 [==============================] - 0s 159us/step - loss: 0.1119 - acc: 0.9608
Epoch 88/100
2067/2067 [==============================] - 0s 147us/step - loss: 0.1153 - acc: 0.9603
Epoch 89/100
2067/2067 [==============================] - 0s 159us/step - loss: 0.1122 - acc: 0.9618
Epoch 90/100
2067/2067 [==============================] - 0s 159us/step - loss: 0.1131 - acc: 0.9603
Epo

## Test the Neural Netwrok on the Test Data


In [33]:
Y_pred = clf_ann.predict(X_test)
Y_pred = (Y_pred > 0.5)

## Check the Accuracy

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(Y_test, Y_pred)

0.9129593810444874

In [35]:
confusion_matrix(Y_test, Y_pred)

array([[460,  19],
       [ 26,  12]], dtype=int64)